<a href="https://colab.research.google.com/github/sokanaid/CourseProject2022BiologicalMath/blob/main/plateau_search_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Результаты поиска плато

In [191]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [192]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/sim_tables

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-192-0e4913ec1a0b>", line 3, in <module>
    get_ipython().run_line_magic('cd', '/content/drive/MyDrive/sim_tables')
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2314, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/magics/osm.py", line 334, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, 

OSError: ignored

## Чтение данных

In [ ]:
def read_csv_file(directory_path, name):
  file_path = os.path.join(directory_path, name + ".csv")
  print("read file ", file_path)
  return pd.read_csv(file_path)

In [ ]:
def read_description(directory_path, file_name = "description.txt", parse_params = False):
  file_path = os.path.join(directory_path, file_name)
  print("read description from file  ", file_path)
  with open(file_path, 'r') as file:
    if not parse_params:
      return file.read()
    result = {}
    for line in file:
      name, value = line.split('=')
      value = value.strip(' \t\n\r')
      if value == 'TRUE':
        value = True
      elif value == 'FALSE':
        value = False
      else:
        try:
          value = float(value)
        except:
          try:
            value = int(value)
          except:
            ...
      result[name] = value
    return result

In [ ]:
def read_files(directory_path):
  data = {}
  # строковое описание параметров
  data["description_str"] = read_description(directory_path)
  # описание параметров в виде словаря
  data["params"] =  read_description(directory_path, parse_params = True)
  # численность популяции
  data["population"] = read_csv_file(directory_path, "population")
  data["population"].drop(data["population"].tail(1).index, inplace=True)
  data["population"].rename(columns={"Unnamed: 0": 'epochs'}, inplace=True)
  # результаты последовательных сглаживаний численности популяции
  name = "exp_pop10"
  data[name] = read_csv_file(directory_path, name)
  data[name].rename(columns={"Unnamed: 0": 'epochs'}, inplace=True)
  return data

In [ ]:
# читаем все результаты симуляций из папок
def read_all_simulations(directories_paths):
  data = []
  for directory_path in directories_paths:
    for simulation_diractory_name in os.listdir(directory_path):
      simulation_diractory_path = os.path.join(directory_path, simulation_diractory_name)
      if os.path.isdir(simulation_diractory_path):
        data.append(read_files(simulation_diractory_path))
  return data

## Построение графиков

## Результаты симуляций

In [ ]:
directories = ["/content/drive/MyDrive/sim_tables/kurkin_b=02_d=002"]
data = read_all_simulations(directories)

In [ ]:
data[0]

In [ ]:
# ищем симуляции на которых не вышли на плато
count = 0
no_plateau_finding = 0
for sim_data in data:
  if sim_data["params"]["auto_stop_at_plateau"] and not sim_data["params"]["found_plateau"]:
    print(sim_data["description_str"])
  elif sim_data["params"]["auto_stop_at_plateau"]:
    count+=1
  else:
    no_plateau_finding+=1

print("successful: ", count)
print("plateau not found: ", len(data) - count - no_plateau_finding)
print("total: ", len(data) - no_plateau_finding)

In [ ]:
# поиск симуляции с теми же параметрами, но без остановки на плато
def find_pair_sim(sim_data, data):
  if not sim_data["params"]["auto_stop_at_plateau"]:
    print(sim_data)
    print("auto_stop_at_plateau = False")
    return
  params = sim_data["params"]
  params.pop("realtime_limit_reached", None)
  params.pop("plateau_threshold", None)
  params["found_plateau"] = False
  params["auto_stop_at_plateau"] = False
  
  pair_sim = None
  for other_sim_data in data:
    is_pair_sim = True
    for param in params:
      if other_sim_data["params"][param]!=params[param]:
        is_pair_sim = False
        break
    if is_pair_sim:
      pair_sim = other_sim_data
      break
  if pair_sim is None:
    print("pair not found params:", sim_data["str_description"])
  return pair_sim

In [ ]:
# найдем симуляции, для которых график без остановки на плато продолжил падать/ расти
big_diff_count =0 
for sim_data in data:
  if not sim_data["params"]["auto_stop_at_plateau"]:
    continue
  # ищем парную симуляцию без остановки на плато
  pair_sim = find_pair_sim(sim_data, data)
  
  dif = int(sim_data["population"].tail(1)['pop'])/int(pair_sim["population"].tail(1)['pop'])
  # TODO: взять среднее за последние 5 или посмотреть на динамику после остановки
  # Проверим результаты вручную,если были выбрано недостаточное количество эпох для парной симуляции
  # или данные парная симуляция начала расти/убывать после предполагаемого выхода на плато
  if len(pair_sim["population"])==len(sim_data["population"]):
    print("less epochs then need. params:", sim_data["description_str"])
    print_pair(sim_data, pair_sim)
  if dif > 1.05 or dif < 0.95:
    print("big diff params:", sim_data["description_str"])
    print_pair(sim_data, pair_sim)
    count+=1
print("count of simulations with ponesial wrong plateau :", big_diff_count)

In [ ]:
data_sim = None
for i, data_p in enumerate(data):
  if data_p["params"]["auto_stop_at_plateau"]:
    data_sim = data_p
    break
print(i)

In [ ]:
data_sim

In [ ]:
def print_pair(data1, data2, x="time"):
  fig = plt.figure(figsize=(20, 10), dpi=80)
  plt.axis('off')
  plt.title("Simulation" )
  fig.tight_layout(pad=6.0,  h_pad=3, w_pad=3)
  fig.patch.set_visible(False)
  axes = fig.subplots(nrows=1, ncols=2)
  for i, data in enumerate([data1, data2]):
    print("i", i)
    print(data)
    print(data["params"]['auto_stop_at_plateau'])
    axes[i].plot(data["population"][x], data["population"]['pop'], label="auto_stop_at_plateau=" +str(data["params"]["auto_stop_at_plateau"]))
    axes[i].legend()
    axes[i].set_xlabel(x)
    axes[i].set_ylabel('pop')

In [ ]:
pair_sim = find_pair_sim(data[0], data)
print("pair_sim", pair_sim)
print_pair(data_sim, pair_sim)